In [208]:
"""    
    Indicador: Rendimientos del Equipo de Reconocimiento para pago
"""

'    \n    Indicador: Rendimientos del Equipo de Reconocimiento para pago\n'

<h3>Importación Librerías</h3>

In [209]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos
import funcion_SQL_a_DataFrame

<h3>Parametrización de Variables</h3>

In [210]:
consulta_tRendimientosActualizacion = """
    select t2.radicado 
        ,t2.usuario_responsable_actividad_anterior
        ,t2.actividad_actual_tramite
        ,t2.usuario_actual_responsable_tramite
        ,t2.con_plano_cartografico
        ,t2.usuario_inserta_plano
        ,t2.npn
        ,t2.municipio
        ,(case when t2.actividad_actual_tramite in ('Consolidando información'
                                                    ,'Incorporando predios a la versión'
                                                    ,'Realizando visita de campo'
                                                    ,'Revisando Información Reconocimiento  Inicial'
                                                    ,'Revisando Información') then 'Reconocimiento'
            when t2.actividad_actual_tramite in ('Realizando Análisis Catastro Registro') then 'Jurídica'
            when t2.actividad_actual_tramite in ('Preparando Edición Geográfica') then 'SIG'
            else 'Sin parametrización'
            end) componente_responsable
        ,(case when actividad_actual_tramite in ('Consolidando información'
                                                ,'Incorporando predios a la versión'
                                                ,'Realizando visita de campo'
                                                ,'Revisando Información Reconocimiento  Inicial'
                                                ,'Realizando Análisis Catastro Registro'
                                                ,'Preparando Edición Geográfica') then 'Fase Previa a la Transacción Catastral'
                when actividad_actual_tramite in ('Revisando Información') then 'En Transacción Catastral o Posterior'
            else 'Sin parametrización'
            end) fase_actividad
            
    from
    (
    select t1.radicado
    ,t1.user_fn_activiad_ant usuario_responsable_actividad_anterior
    ,t1.actividad actividad_actual_tramite
    ,t1.user_actividad_act usuario_actual_responsable_tramite
    ,t1.plano_cartografico con_plano_cartografico
    ,t1.user_inserta_plano usuario_inserta_plano
    ,t1.npn 
    ,t1.municipio 
    from bpmcat.vw_trmactivos_rad_npn_plano t1
    where nom_tramite = 'Formación y Actualización'
    ) t2;
"""

orden_columnas_indicador_2 = ['Incorporando predios a la versión'
    ,'Realizando visita de campo'
    ,'Consolidando información'
    ,'Revisando Información Reconocimiento  Inicial'
]

columnas_croquis = ['municipio'
                    ,'radicado'
                    ,'componente_responsable'
                    ,'npn'
                    ,'usuario_actual_responsable_tramite']

dict_coordinador_reconocedor = {'abel.maldonado':'Coordinador',
                                'angel.banquez':'Reconocedor',
                                'celci.quintana':'Reconocedor',
                                'emilio.castillo':'Reconocedor',
                                'fabian.arango':'Reconocedor',
                                'fabio.barragan':'Reconocedor',
                                'german.panizza':'Reconocedor',
                                'harlingto.padilla':'Reconocedor',
                                'hernando.sarmiento':'Reconocedor',
                                'jacqueline.jimenez':'Coordinador',
                                'jarod.macareno':'Reconocedor',
                                'karen.escobar':'Reconocedor',
                                'luis.jimenez':'Reconocedor',
                                'mary.conquett':'Reconocedor',
                                'ricardo.guerrero':'Reconocedor',
                                'samith.saenz':'Reconocedor',
                                'william.perez':'Reconocedor',
                                'yesmy.batista':'Coordinador',
                                'andres.ramos':'Editor SIG',
                                'melany.delahoz':'Jurídico'
                                }

fecha_actual = datetime.now()
fecha_directorio = str(fecha_actual.strftime("%Y%m%d"))

DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = str(fecha_directorio) + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)

DIRECTORIO_EXCEL_BASE = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\zInsumos"
NOMBRE_EXCEL_BASE = 'base_npn_analisis_seguimiento.xlsx'
RUTA_EXCEL_BASE = os.path.join(DIRECTORIO_EXCEL_BASE, NOMBRE_EXCEL_BASE)

<h3>Ejecución de Funciones</h3>

In [211]:
df_consulta_rendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)

Inicio de la conexión
Conexión exitosa ...


<h3>Incorporación de Rol</h3>

In [212]:
df_consulta_rendimientosActualizacion['Rol'] = None
for llave, valor in dict_coordinador_reconocedor.items():
    df_consulta_rendimientosActualizacion.loc[df_consulta_rendimientosActualizacion['usuario_actual_responsable_tramite']==llave, 'Rol'] = valor

df_consulta_rendimientosActualizacion

,radicado,usuario_responsable_actividad_anterior,actividad_actual_tramite,usuario_actual_responsable_tramite,con_plano_cartografico,usuario_inserta_plano,npn,municipio,componente_responsable,fase_actividad,Rol
0,2979,euclides.polo,Realizando Análisis Catastro Registro,melany.delahoz,Si,euclides.polo,086060002000000000394000000000,REPELÓN,Jurídica,Fase Previa a la Transacción Catastral,Jurídico
1,2753,sandra.acosta,Preparando Edición Geográfica,andres.ramos,No,None,134420000000000020157000000000,MARÍA LA BAJA,SIG,Fase Previa a la Transacción Catastral,Editor SIG
2,2753,sandra.acosta,Preparando Edición Geográfica,andres.ramos,No,None,134420000000000020160000000000,MARÍA LA BAJA,SIG,Fase Previa a la Transacción Catastral,Editor SIG
3,2753,sandra.acosta,Preparando Edición Geográfica,andres.ramos,No,None,134420000000000020161000000000,MARÍA LA BAJA,SIG,Fase Previa a la Transacción Catastral,Editor SIG
4,2753,sandra.acosta,Preparando Edición Geográfica,andres.ramos,No,None,134420000000000020162000000000,MARÍA LA BAJA,SIG,Fase Previa a la Transacción Catastral,Editor SIG
...,...,...,...,...,...,...,...,...,...,...,...
4380,3362,yesmy.batista,Realizando visita de campo,euclides.polo,No,None,086060002000000000389000000000,REPELÓN,Reconocimiento,Fase Previa a la Transacción Catastral,None
4381,3362,yesmy.batista,Realizando visita de campo,euclides.polo,No,None,086060002000000000390000000000,REPELÓN,Reconocimiento,Fase Previa a la Transacción Catastral,None
4382,3362,yesmy.batista,Realizando visita de campo,euclides.polo,No,None,086060002000000000397000000000,REPELÓN,Reconocimiento,Fase Previa a la Transacción Catastral,None
4383,3362,yesmy.batista,Realizando visita de campo,euclides.polo,No,None,086060002000000000398000000000,REPELÓN,Reconocimiento,Fase Previa a la Transacción Catastral,None


<h3>1. Análisis Rendimientos sobre Reconocimiento</h3>
<ol>
    <li> Indicador 1: Total de Predios por Fase de Actividad y Reconocedor
    <li> Indicador 2: Total de predios, fase previa a Transacción, por usuario
    <li> Indicador 3: Total de predios, fase Transacción o posterior, por usuario
    <li> Indicador 4: Total de predios, que fueron liberados por reconocimiento previo análisis SIG
</ol>

In [213]:

# ? Indicador 1
df_rendActua_ERecono = df_consulta_rendimientosActualizacion[df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento']
df_rendActua_ERecono_indicador_1 = pd.DataFrame(df_rendActua_ERecono.groupby(['municipio','fase_actividad','Rol'])['usuario_actual_responsable_tramite'].value_counts()).reset_index().rename(columns={'count':'total_predios'})
print(f"Indicador 1: \n {df_rendActua_ERecono_indicador_1}")

# ? Indicador 2
df_rendActua_ERecono_FasePrevia = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento') & (df_consulta_rendimientosActualizacion['fase_actividad']=='Fase Previa a la Transacción Catastral')]

# TODO: Se crea diccionario para ordenar columnas de acuerdo a flujo de trabajo de Actualización
dict_orden_columnas_ind_2 = {valor: index for index, valor in enumerate(orden_columnas_indicador_2)}

df_rendActua_ERecono_indicador_2 = pd.DataFrame(df_rendActua_ERecono_FasePrevia.groupby(['municipio','actividad_actual_tramite','Rol'])['usuario_actual_responsable_tramite'].value_counts()).reset_index().rename(columns={'count':'total_predios'})
print(f"Indicador 2: \n {df_rendActua_ERecono_indicador_2}")

# # TODO: Ordenar el DataFrame usando el diccionario de mapeo
df_rendActua_ERecono_indicador_2['actividad_ordenada_s_flujo'] = df_rendActua_ERecono_indicador_2['actividad_actual_tramite'].map(dict_orden_columnas_ind_2)
df_rendActua_ERecono_indicador_2_Ordenado = df_rendActua_ERecono_indicador_2.sort_values(by='actividad_ordenada_s_flujo').drop(columns='actividad_ordenada_s_flujo')

# ? Indicador 3
df_rendActua_ERecono_FaseTransaccionOPost = df_consulta_rendimientosActualizacion[(df_consulta_rendimientosActualizacion['componente_responsable']=='Reconocimiento') & 
                                                                                  (df_consulta_rendimientosActualizacion['fase_actividad']=='En Transacción Catastral o Posterior')]

df_rendActua_ERecono_indicador_3 = pd.DataFrame(df_rendActua_ERecono_FaseTransaccionOPost.groupby(['municipio','actividad_actual_tramite','Rol'])['usuario_responsable_actividad_anterior'].value_counts()).reset_index().rename(columns={'count':'total_predios'})
print(f"Indicador 3 \n {df_rendActua_ERecono_indicador_3}")

Indicador 1: 
         municipio                          fase_actividad          Rol  \
0   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
1   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
2   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
3   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
4   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
5   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
6   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
7   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
8   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
9   MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
10  MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
11  MARÍA LA BAJA  Fase Previa a la Transacción Catastral  Reconocedor   
12  MARÍA LA BAJA  Fase

In [214]:

# ? Indicador 4
df_PrediosPreparandoEdicionGeo = df_consulta_rendimientosActualizacion[df_consulta_rendimientosActualizacion['actividad_actual_tramite'] == 'Preparando Edición Geográfica']

df_PrediosPreparandoEdicionGeo_Base = df_PrediosPreparandoEdicionGeo[['radicado', 'municipio', 'usuario_responsable_actividad_anterior', 'npn']]

if os.path.exists(RUTA_EXCEL_BASE):
    print(f"El archivo {NOMBRE_EXCEL_BASE} ya existe")
else:
    df_PrediosPreparandoEdicionGeo_Base.to_excel(RUTA_EXCEL_BASE)
    print(f"Se crea el archivo base {NOMBRE_EXCEL_BASE}")

df_lectura_xlsx_base = pd.read_excel(RUTA_EXCEL_BASE)

nombre_columna = 'total_predios_' + str(fecha_directorio)

df_PrediosPreparandoEdicionGeo_Indicador_4 = pd.DataFrame(df_lectura_xlsx_base.groupby(['municipio'])['usuario_responsable_actividad_anterior'].value_counts()).reset_index().rename(columns={'count':nombre_columna})
df_PrediosPreparandoEdicionGeo_Indicador_4

El archivo base_npn_analisis_seguimiento.xlsx ya existe


,municipio,usuario_responsable_actividad_anterior,total_predios_20240617
0,MARÍA LA BAJA,rafael.molina,244
1,MARÍA LA BAJA,mary.conquett,121
2,MARÍA LA BAJA,denilson.correa,78
3,MARÍA LA BAJA,angel.banquez,76
4,MARÍA LA BAJA,nadith.miranda,65
5,MARÍA LA BAJA,sandra.acosta,61
6,MARÍA LA BAJA,maria.hernandez,55
7,MARÍA LA BAJA,emilio.castillo,44
8,MARÍA LA BAJA,hernando.sarmiento,11


<h3>Indicadores 1, 2 y 3 para Tabla XLSX<h3>

In [215]:

# ? Indicador 1
df_rendActua_ERecono_indicador_1_pivote = df_rendActua_ERecono_indicador_1.pivot(index='usuario_actual_responsable_tramite', columns='fase_actividad', values='total_predios')

for columna in df_rendActua_ERecono_indicador_1_pivote.columns:
    df_rendActua_ERecono_indicador_1_pivote.loc[df_rendActua_ERecono_indicador_1_pivote[columna].isnull(), columna] = 0
    df_rendActua_ERecono_indicador_1_pivote[columna] = df_rendActua_ERecono_indicador_1_pivote[columna].astype(int)

df_rendActua_ERecono_indicador_1_pivote = pd.DataFrame(df_rendActua_ERecono_indicador_1_pivote).reset_index()

# ? Indicador 2
df_rendActua_ERecono_indicador_2_Ordenado_pivote = df_rendActua_ERecono_indicador_2_Ordenado.pivot(index='usuario_actual_responsable_tramite', columns='actividad_actual_tramite', values='total_predios')
df_rendActua_ERecono_indicador_2_Ordenado_pivote =df_rendActua_ERecono_indicador_2_Ordenado_pivote[orden_columnas_indicador_2]

for columna in df_rendActua_ERecono_indicador_2_Ordenado_pivote.columns:
    df_rendActua_ERecono_indicador_2_Ordenado_pivote.loc[df_rendActua_ERecono_indicador_2_Ordenado_pivote[columna].isnull(), columna] = 0
    df_rendActua_ERecono_indicador_2_Ordenado_pivote[columna] = df_rendActua_ERecono_indicador_2_Ordenado_pivote[columna].astype(int)

df_rendActua_ERecono_indicador_2_Ordenado_pivote = pd.DataFrame(df_rendActua_ERecono_indicador_2_Ordenado_pivote).reset_index()

# ? Indicador 3
df_rendActua_ERecono_indicador_3_pivote = df_rendActua_ERecono_indicador_3.pivot(index='usuario_responsable_actividad_anterior', columns='actividad_actual_tramite', values='total_predios')

for columna in df_rendActua_ERecono_indicador_3_pivote.columns:
    df_rendActua_ERecono_indicador_3_pivote.loc[df_rendActua_ERecono_indicador_3_pivote[columna].isnull(), columna] = 0
    df_rendActua_ERecono_indicador_3_pivote[columna] = df_rendActua_ERecono_indicador_3_pivote[columna].astype(int)

df_rendActua_ERecono_indicador_3_pivote = pd.DataFrame(df_rendActua_ERecono_indicador_3_pivote).reset_index()


<h3>Reporte Croquis Vs NPN</h3>

In [216]:
df_npn_croquis = df_consulta_rendimientosActualizacion[df_consulta_rendimientosActualizacion['con_plano_cartografico']=='Si']
df_npn_croquis_estandarizado = df_npn_croquis[columnas_croquis]

<h3>Generación XLSX</h3>

In [217]:

# ** Usar ExcelWriter para guardar múltiples dataframes en diferentes hojas
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='xlsxwriter') as writer:
    df_rendActua_ERecono_indicador_1_pivote.to_excel(writer, sheet_name='Indicador_1', index=False)
    df_rendActua_ERecono_indicador_2_Ordenado_pivote.to_excel(writer, sheet_name='Indicador_2', index=False)
    df_rendActua_ERecono_indicador_3_pivote.to_excel(writer, sheet_name='Indicador_3', index=False)
    df_npn_croquis_estandarizado.to_excel(writer, sheet_name='Reporte_NPN_Croquis', index=False)

print(f"Se exporta XLSX {NOMBRE_XLSX}, en la ruta {DIRECTORIO_SALIDA_XLSX}")


PermissionError: [Errno 13] Permission denied: 'C:\\docsProyectos\\5.RAISS\\2024.0.RAISS_Lote_4\\6.Hitos\\E2_Informes_Id_FisicoJuridica\\2_2_2_Rendimientos_Reconocimiento\\20240617_RendimientoEquipoReconocimiento_BCGS.xlsx'